In [1]:
import dask
import dask.threaded
import dask.multiprocessing
from dask.distributed import Client, LocalCluster

cluster = LocalCluster(n_workers=2, threads_per_worker=2,dashboard_address=':8888')

client = Client(cluster)
client

Client Scheduler: tcp://127.0.0.1:39953 Dashboard: http://127.0.0.1:8888/status,Cluster Workers: 2 Cores: 4 Memory: 35.01 GB


In [2]:
## Path for modules

import sys

import numpy as np
import xarray as xr

from netCDF4 import Dataset

import numpy.ma as ma

sys.path.insert(0,"/mnt/meom/workdir/henelle/Notebooks/git/")
import myfunctions as mf



import matplotlib.pyplot as plt
import matplotlib as mpl
from xhistogram.xarray import histogram

import pandas as pd

%matplotlib inline

### 1. Opening curl & strain files

In [3]:
season = 'JAS'

In [4]:
# With tide

dirvarT0     = '/mnt/meom/workdir/alberta/eNATL60/eNATL60-BLBT02-S/1h/ACO/'
curlfileT0   = dirvarT0 + 'eNATL60ACO-BLBT02_y2009m07-09_socurloverf10m_filt2T.nc' # JAS
strainfileT0 = dirvarT0 + 'eNATL60ACO-BLBT02_y2009m07-09_sostrainoverf10m_filt2T.nc' # JAS

#curlfileT0   = dirvarT0 + 'eNATL60ACO-BLBT02_y2010m01-03_socurloverf10m_filt2T.nc' # JFM
#strainfileT0 = dirvarT0 + 'eNATL60ACO-BLBT02_y2010m01-03_sostrainoverf10m_filt2T.nc' # JFM


dscurlT0   = xr.open_mfdataset(curlfileT0,  combine='by_coords',parallel=True,chunks={'x':200,'y':200})
dsstrainT0 = xr.open_mfdataset(strainfileT0,combine='by_coords',parallel=True,chunks={'x':200,'y':200})

curlT0   = dscurlT0.Curl_Tide_Filt # JAS
#curlT0   = dscurlT0.curl_Tide_Filt # JFM
strainT0 = dsstrainT0.strain_Tide_Filt

In [5]:
# No tide

dirvar00     = '/mnt/meom/workdir/alberta/eNATL60/eNATL60-BLB002-S/1h/ACO/'
curlfile00   = dirvar00 + 'eNATL60ACO-BLB002_y2009m07-09_socurloverf10m_filt2T.nc' # JAS
strainfile00 = dirvar00 + 'eNATL60ACO-BLB002_y2009m07-09_sostrainoverf10m_filt2T.nc' # JAS

#curlfile00   = dirvar00 + 'eNATL60ACO-BLB002_y2010m01-03_socurloverf10m_filt2T.nc' # JFM
#strainfile00 = dirvar00 + 'eNATL60ACO-BLB002_y2010m01-03_sostrainoverf10m_filt2T.nc' # JFM


dscurl00   = xr.open_mfdataset(curlfile00,  combine='by_coords',parallel=True,chunks={'x':200,'y':200})
dsstrain00 = xr.open_mfdataset(strainfile00,combine='by_coords',parallel=True,chunks={'x':200,'y':200})

curl00   = dscurl00.Curl_noTide_Filt # JAS
#curl00   = dscurl00.curl_noTide_Filt # JFM
strain00 = dsstrain00.strain_noTide_Filt

print(curl00.shape)

(2208, 1191, 781)


In [6]:
# Opening grid files

dirgrid = '/mnt/meom/workdir/alberta/eNATL60/eNATL60-I/'
gridfile = dirgrid+'mesh_hgr_eNATL60ACO_3.6.nc'
dsgrid = xr.open_dataset(gridfile,chunks={'x':200,'y':200})

lon = dsgrid.nav_lon
lat = dsgrid.nav_lat


In [7]:
lon.shape

(1191, 781)

### 2. Choosing region of interest

In [8]:
#lonmin, lonmax, latmin, latmax = -31,-28,30.6,33.6 # region A3
#lonmin, lonmax, latmin, latmax = -31,-28,31.6,34.6 # region A1
#lonmin, lonmax, latmin, latmax = -31,-28,33,36 # region B
#lonmin, lonmax, latmin, latmax = -30,-27,30.6,33.6 # region A2
#lonmin, lonmax, latmin, latmax = -35.0,-32.5,35.4,39.0 # region C
lonmin, lonmax, latmin, latmax = -34.0,-31.5,25.2,28.8 # region D

region = 'D'

box = (lonmin, lonmax, latmin, latmax)

domain = (box[0]<lon)*(lon<box[1])*(box[2]<lat)*(lat<box[3])
where = np.where(domain)

In [9]:
jmin = where[0][0]
jmax = where[0][-1]
imin = where[1][0]
imax = where[1][-1]

In [10]:
print(imin,imax,jmin,jmax)

205 354 134 375


In [11]:
strainT0_filtbox = strainT0[:,jmin:jmax+1,imin:imax+1]
strain00_filtbox = strain00[:,jmin:jmax+1,imin:imax+1]
curlT0_filtbox   = curlT0  [:,jmin:jmax+1,imin:imax+1]
curl00_filtbox   = curl00  [:,jmin:jmax+1,imin:imax+1]

### 3. Defining bins and weights for histograms

In [12]:
sbin_width = 0.01 # bin widths tested: 0.05 original, 0.025, 0.1, 0.0125
cbin_width = 0.02  # bin widths tested: 0.1 original, 0.05, 0.2, 0.025

In [13]:
sbins = np.arange(0.0,1.0, sbin_width)
cbins = np.arange(-1.0,1.0, cbin_width)

In [14]:
number_sbins = (1.0 - 0.0) / sbin_width
number_cbins = (1.0 - (-1.0)) / cbin_width
print('Strain has '+str(int(number_sbins))+' bins.')
print('Curl has '+str(int(number_cbins))+' bins.')

Strain has 100 bins.
Curl has 100 bins.


In [15]:
strain00_filtbox.name ='strain00'
strainT0_filtbox.name ='strainT0'
curl00_filtbox.name   ='curl00'
curlT0_filtbox.name   ='curlT0'

In [16]:
histo2D_00 = histogram(strain00_filtbox,curl00_filtbox, bins=[sbins, cbins])
histo2D_T0 = histogram(strainT0_filtbox,curlT0_filtbox, bins=[sbins, cbins])

In [17]:
# Convert tuple to int
nt = int(''.join(map(str, strainT0_filtbox[:,0,0].shape)))
ny = int(''.join(map(str, strainT0_filtbox[0,:,0].shape)))
nx = int(''.join(map(str, strainT0_filtbox[0,0,:].shape)))
print(nt,ny,nx)

2208 242 150


In [18]:
vol = nt * ny * nx
print(vol)

80150400


In [19]:
# Normalizing through the division by the area
histo2D_00_norm2 = ( histo2D_00 / vol ) / ( sbin_width * cbin_width )
histo2D_T0_norm2 = ( histo2D_T0 / vol ) / ( sbin_width * cbin_width )

#histo2D_00_norm2 = histo2D_00 / ( sbin_width * cbin_width )
#histo2D_T0_norm2 = histo2D_T0 / ( sbin_width * cbin_width )

In [20]:
histo2D_00_norm2.shape

(99, 99)

### 4. Saving 2D histograms

In [21]:
new_filename_1 = './histo2D_00_norm2'+region+season+'_1.nc'
print ('saving to ', new_filename_1)
histo2D_00_norm2.to_netcdf(path=new_filename_1)
histo2D_00_norm2.close()
print ('finished saving')

saving to  ./histo2D_00_norm2DJAS_1.nc
finished saving


In [22]:
new_filename_2 = './histo2D_T0_norm2'+region+season+'_1.nc'
print ('saving to ', new_filename_2)
histo2D_T0_norm2.to_netcdf(path=new_filename_2)
histo2D_T0_norm2.close()
print ('finished saving')

saving to  ./histo2D_T0_norm2DJAS_1.nc
finished saving
